In [1]:
from multiversx_sdk_wallet import Mnemonic
from multiversx_sdk_core import Address
from multiversx_sdk_core import AddressFactory

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

['walnut', 'allow', 'elegant', 'raise', 'jungle', 'display', 'door', 'tiger', 'vivid', 'toe', 'knock', 'uncle', 'talent', 'winner', 'math', 'alter', 'suffer', 'evoke', 'wave', 'smart', 'foster', 'civil', 'future', 'bus']


In [11]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print(secret_key.hex())
print("Public key (hex-encoded):", secret_key.hex())

19b9cf83c67a1c21ed313ac4d75e66df71d267a15dcf609f37ca5695ee86bb7f
Public key (hex-encoded): 19b9cf83c67a1c21ed313ac4d75e66df71d267a15dcf609f37ca5695ee86bb7f


In [16]:
factory = AddressFactory("erd")
address = factory.create_from_hex("19b9cf83c67a1c21ed313ac4d75e66df71d267a15dcf609f37ca5695ee86bb7f")

In [9]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [11]:
result = supabase.table('chatbot_messages') \
  .insert({
    "user_id": "123456789",
    "chat_id": "chat_001",
    "text": "Hello, how can I help you?",
    "is_bot": True
  }) \
  .execute()
print(result)

2023-10-05 13:49:10,245:INFO - HTTP Request: POST https://dfdobpmlckedsihyzyov.supabase.co/rest/v1/chatbot_messages "HTTP/1.1 201 Created"


data=[{'message_id': 2, 'user_id': '123456789', 'chat_id': 'chat_001', 'text': 'Hello, how can I help you?', 'is_bot': True, 'timestamp': '2023-10-05T08:19:11.06578'}] count=None


## Code Generation

In [31]:
# from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
# Clone
repo_path = "./github-scrapper/code_scrapped"

In [33]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".rs"],
    parser=LanguageParser(language=Language.RUST, parser_threshold=500)
)
documents = loader.load()
len(documents)

38

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
rust_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.RUST, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = rust_splitter.split_documents(documents)
len(texts)

105

In [35]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [42]:
question = "Can you generate me a contract code that contains the number 8 and i can increase it?"
result = qa(question)
result['answer']

'Sure! Here\'s a contract code that includes the number 8 and allows for its increase:\n\n```rust\n#![no_std]\n\nmultiversx_sc::imports!();\n\n/// A contract that holds the number 8 and allows for its increase.\n#[multiversx_sc::contract]\npub trait NumberIncrementer {\n    #[storage_mapper("number")]\n    fn number(&self) -> SingleValueMapper<u32>;\n\n    #[init]\n    fn init(&self) {\n        self.number().set(8);\n    }\n\n    /// Increases the stored number by a given amount.\n    #[endpoint]\n    fn increase_number(&self, amount: u32) {\n        self.number().update(|num| *num += amount);\n    }\n}\n```\n\nThis contract initializes the storage value `number` to 8 using the `init` function. It also provides an `increase_number` endpoint that takes an `amount` parameter and increases the stored number by that amount.'

In [43]:
print(result['answer'])

Sure! Here's a contract code that includes the number 8 and allows for its increase:

```rust
#![no_std]

multiversx_sc::imports!();

/// A contract that holds the number 8 and allows for its increase.
#[multiversx_sc::contract]
pub trait NumberIncrementer {
    #[storage_mapper("number")]
    fn number(&self) -> SingleValueMapper<u32>;

    #[init]
    fn init(&self) {
        self.number().set(8);
    }

    /// Increases the stored number by a given amount.
    #[endpoint]
    fn increase_number(&self, amount: u32) {
        self.number().update(|num| *num += amount);
    }
}
```

This contract initializes the storage value `number` to 8 using the `init` function. It also provides an `increase_number` endpoint that takes an `amount` parameter and increases the stored number by that amount.


In [ ]:
print(result['answer'])

Here's an example of a contract that has a storage variable initialized with the number 8 and provides an endpoint to increase it:

```rust
#![no_std]

multiversx_sc::imports!();

#[multiversx_sc::contract]
pub trait IncrementContract {
    #[init]
    fn init(&self) {
        self.counter().set(8u64);
    }

    #[endpoint]
    fn increase(&self, value: u64) {
        self.counter().update(|counter| *counter += value);
    }

    #[storage_mapper("counter")]
    fn counter(&self) -> SingleValueMapper<Self::Storage, u64>;
}
```

In this contract, there is an `init` function that sets the `counter` to 8. Then, there's an `increase` endpoint function that adds a given value to the `counter`. The `counter` is stored persistently in the contract's storage.
